In [ ]:
import wandb

In [ ]:
!wandb login


In [ ]:
import wandb
import os
os.environ['WANDB_DIR'] = os.getcwd() + '/wandb/'
os.environ['WANDB_CACHE_DIR'] = os.getcwd() + '/wandb/.cache/'
os.environ['WANDB_CONFIG_DIR'] = os.getcwd() + '/wandb/.config/'

import torch
import torch.nn as nn
import torch.optim as optim

# 初始化wandb
wandb.init(
    entity='kaifan-li',
    project="my_project"
)

# 构建模型
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(10, 1)
    
    def forward(self, x):
        return self.fc(x)

model = SimpleModel()

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 训练循环
for epoch in range(10):
    running_loss = 0.0
    for i in range(100):
        inputs = torch.randn(32, 10)  # 随机生成输入数据
        labels = torch.randn(32, 1)   # 随机生成标签
        
        # 正向传播
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # 记录训练过程和指标
    avg_loss = running_loss / 100
    wandb.log({"epoch": epoch, "loss": avg_loss})


----

In [3]:
from model.modeling_bart import BartPrefixPropForConditionalGeneration
from transformers import BartTokenizerFast, BartConfig, AutoConfig
from config.custom_config import PromptBartConfig
config = AutoConfig.from_pretrained('facebook/bart-base')
custom_config = PromptBartConfig(
    **config.to_dict(),
)
model = BartPrefixPropForConditionalGeneration.from_pretrained('facebook/bart-base', config=custom_config)
model

Some weights of BartPrefixPropForConditionalGeneration were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['propagated_prefix_encoder.embedding.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BartPrefixPropForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((7

In [4]:
from transformers import AutoTokenizer
text = 'i love you'
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-base')
input_ids = tokenizer(text, return_tensors='pt')['input_ids']
model.generate(
    input_ids,
    test_flag='this is a test'
)

test_flag='this is a test'
self.config.pre_seq_len=20
this is a flag


/home/is/kaifan-l/miniconda3/envs/summarization/lib/python3.11/site-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


AttributeError: 'NoneType' object has no attribute 'shape'

In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('facebook/bart-base')

In [3]:
data = [
    [['i love you'], ['this is a test']],
    [['i love you'], ['this is a test']],
]

input_ids = tokenizer(data, return_tensors='pt', padding=True, truncation=True)
input_ids

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [11]:
from datasets import load_dataset

dataset = load_dataset('json', data_dir='datasets/pubmed-dataset/')
dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/is/kaifan-l/.cache/huggingface/datasets/json/default-0bcfcacc1ac8bbee/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['article_id', 'article_text', 'abstract_text', 'labels', 'section_names', 'sections'],
        num_rows: 119924
    })
    validation: Dataset({
        features: ['article_id', 'article_text', 'abstract_text', 'labels', 'section_names', 'sections'],
        num_rows: 6633
    })
    test: Dataset({
        features: ['article_id', 'article_text', 'abstract_text', 'labels', 'section_names', 'sections'],
        num_rows: 6658
    })
})

In [28]:
sections_0 = dataset['train']['sections'][0]

In [37]:
section_names_0 = dataset['train']['section_names'][0]
section_names_0

['INTRODUCTION',
 'MATERIALS AND METHODS',
 'Participants',
 'Instruments',
 'Procedure',
 'First step',
 'Second step',
 'Third step',
 'Forth step',
 'Interventions',
 'Fifth step (assessment)',
 'Data analysis',
 'RESULTS',
 'DISCUSSION',
 'CONCLUSION']

In [54]:
article_text_0 = dataset['train']['article_text'][0]
article_text_0

['a recent systematic analysis showed that in 2011 , 314 ( 296 - 331 ) million children younger than 5 years were mildly , moderately or severely stunted and 258 ( 240 - 274 ) million were mildly , moderately or severely underweight in the developing countries .',
 'in iran a study among 752 high school girls in sistan and baluchestan showed prevalence of 16.2% , 8.6% and 1.5% , for underweight , overweight and obesity , respectively .',
 'the prevalence of malnutrition among elementary school aged children in tehran varied from 6% to 16% .',
 'anthropometric study of elementary school students in shiraz revealed that 16% of them suffer from malnutrition and low body weight .',
 'snack should have 300 - 400 kcal energy and could provide 5 - 10 g of protein / day . nowadays , school nutrition programs are running as the national programs , world - wide . national school lunch program in the united states',
 'there are also some reports regarding school feeding programs in developing cou

In [86]:
for sec in (dataset['train']['sections'][:10]):
    print(sec)

[['a recent systematic analysis showed that in 2011 , 314 ( 296 - 331 ) million children younger than 5 years were mildly , moderately or severely stunted and 258 ( 240 - 274 ) million were mildly , moderately or severely underweight in the developing countries .', 'in iran a study among 752 high school girls in sistan and baluchestan showed prevalence of 16.2% , 8.6% and 1.5% , for underweight , overweight and obesity , respectively .', 'the prevalence of malnutrition among elementary school aged children in tehran varied from 6% to 16% .', 'anthropometric study of elementary school students in shiraz revealed that 16% of them suffer from malnutrition and low body weight .', 'snack should have 300 - 400 kcal energy and could provide 5 - 10 g of protein / day . nowadays , school nutrition programs are running as the national programs , world - wide . national school lunch program in the united states', 'there are also some reports regarding school feeding programs in developing countri

In [87]:
dataset['train']['sections'][:10]
# TODO: 需要把最里面一层的句子合并成一个句子，然后再进行分词

[[['a recent systematic analysis showed that in 2011 , 314 ( 296 - 331 ) million children younger than 5 years were mildly , moderately or severely stunted and 258 ( 240 - 274 ) million were mildly , moderately or severely underweight in the developing countries .',
   'in iran a study among 752 high school girls in sistan and baluchestan showed prevalence of 16.2% , 8.6% and 1.5% , for underweight , overweight and obesity , respectively .',
   'the prevalence of malnutrition among elementary school aged children in tehran varied from 6% to 16% .',
   'anthropometric study of elementary school students in shiraz revealed that 16% of them suffer from malnutrition and low body weight .',
   'snack should have 300 - 400 kcal energy and could provide 5 - 10 g of protein / day . nowadays , school nutrition programs are running as the national programs , world - wide . national school lunch program in the united states',
   'there are also some reports regarding school feeding programs in de

In [59]:
sections_0[0]

['a recent systematic analysis showed that in 2011 , 314 ( 296 - 331 ) million children younger than 5 years were mildly , moderately or severely stunted and 258 ( 240 - 274 ) million were mildly , moderately or severely underweight in the developing countries .',
 'in iran a study among 752 high school girls in sistan and baluchestan showed prevalence of 16.2% , 8.6% and 1.5% , for underweight , overweight and obesity , respectively .',
 'the prevalence of malnutrition among elementary school aged children in tehran varied from 6% to 16% .',
 'anthropometric study of elementary school students in shiraz revealed that 16% of them suffer from malnutrition and low body weight .',
 'snack should have 300 - 400 kcal energy and could provide 5 - 10 g of protein / day . nowadays , school nutrition programs are running as the national programs , world - wide . national school lunch program in the united states',
 'there are also some reports regarding school feeding programs in developing cou

In [31]:
abstract_0 = dataset['train']['abstract_text'][0]

In [58]:
# TODO: abstract需要重新分层
dataset['train']['abstract_text'][0]

['<S> background : the present study was carried out to assess the effects of community nutrition intervention based on advocacy approach on malnutrition status among school - aged children in shiraz , iran.materials and methods : this case - control nutritional intervention has been done between 2008 and 2009 on 2897 primary and secondary school boys and girls ( 7 - 13 years old ) based on advocacy approach in shiraz , iran . </S>',
 '<S> the project provided nutritious snacks in public schools over a 2-year period along with advocacy oriented actions in order to implement and promote nutritional intervention . for evaluation of effectiveness of the intervention growth monitoring indices of pre- and post - intervention were statistically compared.results:the frequency of subjects with body mass index lower than 5% decreased significantly after intervention among girls ( p = 0.02 ) . </S>',
 '<S> however , there were no significant changes among boys or total population . </S>',
 '<S> 

In [ ]:
sections = tokenizer(dataset['train'][0]['sections'], return_tensors='pt', padding=True, truncation=True)

In [53]:
print(dataset['train']['labels'])

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non

In [70]:
input_ids = tokenizer(dataset['train']['abstract_text'][0], return_tensors='pt', padding=True, truncation=True)
input_ids['input_ids'].shape

torch.Size([6, 91])

In [65]:
type(input_ids)

transformers.tokenization_utils_base.BatchEncoding

In [124]:
text_column = 'sections'
summary_column = 'abstract_text'
padding = 'max_length'
def preprocess_function_for_section_structure(examples):
    # print(examples['sections'])
    # raise ValueError
    inputs, targets = [], []
    # filter out None samples
    # print(f'{examples[text_column]=}')
    for i in range(len(examples[text_column])):
        if examples[text_column][i] and examples[summary_column][i]:
            # print(f'{examples[text_column][i]=}')
            inputs.append(examples[text_column][i])
            # print(inputs[i])
            targets.append(examples[summary_column][i])
    
    model_inputs = {
        'input_ids': [],
        'attention_mask': [],
        'labels': [],
    }
    # 'sections':
    # [ 第n篇文章/第n行
        # [ # 第n篇文章的第n个section
            # [seq1, seq2, seq3, ...], 
            # [seq1, seq2, seq3, ...], 
            # [seq1, seq2, seq3, ...]
        # ], 
        # [
            # [section1], 
            # [section2], 
            # [section3]
            # ]
    # ]
    section_inputs_ids = []
    section_attention_mask = []
    for text in inputs: # text: 第n篇文章/第n行
        for section in text: # section: 第n篇文章的第n个section
            # print(f'{section=}')
            sec_inputs = tokenizer(section, max_length=1024, padding='max_length', truncation=True)
            # print(sec_inputs) [section1], [seq1, seq2, seq3], [section3]
            # raise ValueError
            section_inputs_ids.append(sec_inputs['input_ids'])
            section_attention_mask.append(sec_inputs['attention_mask'])
        model_inputs['input_ids'].append(section_inputs_ids)
        model_inputs['attention_mask'].append(section_attention_mask)
        
    for targ in targets: 
        row_targets = tokenizer(text_target=targ, max_length=128, padding='max_length', truncation=True)
        # print(row_targets)
        model_inputs['labels'].append(row_targets['input_ids'])
        
    # model_inputs = tokenizer(inputs, max_length=1024, padding='max_length', truncation=True)
    # labels = tokenizer(text_target=targets, max_length=128, padding='max_length', truncation=True)
    # TODO: 这里的labels由于也是多个句子, 所以判断条件需要更爱
    for i in range(len(model_inputs['labels'])):
        model_inputs['labels'][i] = [l if l != tokenizer.pad_token_id else -100 for l in model_inputs['labels'][i]]
    
    return model_inputs

In [125]:
model_outputs = preprocess_function_for_section_structure(dataset['train'][:2])
model_outputs['labels']

[[[0,
   41552,
   104,
   15698,
   3618,
   4832,
   5,
   1455,
   892,
   21,
   2584,
   66,
   7,
   7118,
   5,
   3038,
   9,
   435,
   10894,
   6530,
   716,
   15,
   8973,
   1548,
   15,
   29638,
   2194,
   566,
   334,
   111,
   5180,
   408,
   11,
   1481,
   853,
   1222,
   2156,
   10209,
   260,
   4,
   36739,
   29,
   8,
   6448,
   4832,
   42,
   403,
   111,
   797,
   22813,
   6530,
   34,
   57,
   626,
   227,
   2266,
   8,
   2338,
   15,
   971,
   6750,
   2270,
   8,
   5929,
   334,
   2786,
   8,
   1972,
   36,
   262,
   111,
   508,
   107,
   793,
   4839,
   716,
   15,
   8973,
   1548,
   11,
   1481,
   853,
   1222,
   2156,
   10209,
   260,
   479,
   49703,
   104,
   15698,
   2,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1],
  [0,
   41552,
   104,
   15698,


In [5]:
seqs = ['i love you', 'i love you', 'i love you']

seq = ".".join(seqs)
seq, seqs

('i love you.i love you.i love you',
 ['i love you', 'i love you', 'i love you'])